# Pattern mining

In [2]:
# https://data-mining.philippe-fournier-viger.com/introduction-to-sequential-rule-mining/

In [3]:
# %pip install awswrangler
# %pip install apyori
# %pip install pyfpgrowth

In [4]:
import os 
import json
import boto3 
import random
#import pyfpgrowth
import numpy as np
import pandas as pd
import awswrangler as wr
from apyori import apriori

from sagemaker import get_execution_role
role = get_execution_role()

In [6]:
selected_region='cosr-s'
bucket = 'ons-dl-dev-landing'
prefix = 'ons/mpo_ds_nlp/data/postprocessed_output/comprehend/sentences/entities-cosr-nco-comprehend-baseline.parquet'
input_uri = 's3://{}/{}'.format(bucket, prefix)
df = pd.read_parquet(input_uri)
df

,text,entity,type,score
0,Desenergização da LT 230 kV Tucuruí / Altamira...,"[Desenergização, LT 230 kV Tucuruí / Altamira,...","[ACAO_A_EXECUTAR, EQUIPAMENTO, ACAO_A_EXECUTAR]","[0.9999902248382568, 1.0, 0.9997515678405762]"
1,ProcedimentosObjetivo / Item de Controle1---CO...,"[Verificar, indisponibilidade, LT 230 kV Tucur...","[ACAO_A_EXECUTAR, ESTADO_OPERATIVO, EQUIPAMENTO]","[0.9999995231628418, 0.9999918937683105, 0.999..."
2,2COSR-NCOCOSR-NCOEquatorialDesligar o religame...,"[religamento automático, LT 230 kV Altamira / ...","[ACAO_A_EXECUTAR, EQUIPAMENTO]","[0.9972758293151855, 0.9999115467071533]"
3,3COSR-NCOCOSR-NCOEletronorteAbrir o terminal d...,"[o terminal, LT 230 kV Tucuruí / Altamira,, SE...","[ACAO_A_EXECUTAR, EQUIPAMENTO, SUBESTACAO]","[0.9999996423721313, 1.0, 0.9858415722846985]"
4,- Reator da LT disponível:SE Tucuruí ≤ 232 kV-...,"[Reator da LT, SE Tucuruí, 232 kV-, Reator da ...","[EQUIPAMENTO, SUBESTACAO, VALOR_COM_UNID.MEDID...","[0.9989086389541626, 0.9999986290931702, 0.780..."
...,...,...,...,...
380,V ≤ 46 kV  ≤ 2003COSR-NCOCOSR-NCOCOS-Am GT (A...,"[46 kV, Amazonas, GT Normalizar o atendimento ...","[VALOR_COM_UNID.MEDIDA, SUBESTACAO, ACAO_A_EXE...","[0.9999377131462097, 0.7745931148529053, 0.999..."
381,4COSR-NCOCOSR-NCOCOS-Am GT (Amazonas GT Ligar ...,"[Ligar o religamento automático, LT 230 kV Bal...","[ACAO_A_EXECUTAR, EQUIPAMENTO]","[0.8638736009597778, 0.9999288320541382]"
382,Desenergização da LT 230 kV Cristiano Rocha / ...,"[Desenergização, LT 230 kV Cristiano Rocha / L...","[ACAO_A_EXECUTAR, EQUIPAMENTO, ACAO_A_EXECUTAR...","[0.9993795156478882, 0.9999955296516418, 0.816..."
383,- Atendida as limitações impostas pela indispo...,"[Atendida, indisponibilidade, LT 230 kV Cristi...","[ACAO_A_EXECUTAR, ESTADO_OPERATIVO, EQUIPAMENT...","[0.9999533891677856, 0.9999958276748657, 0.999..."


In [7]:
df = df[df['entity'].map(lambda d: len(d)) > 0]
df

,text,entity,type,score
0,Desenergização da LT 230 kV Tucuruí / Altamira...,"[Desenergização, LT 230 kV Tucuruí / Altamira,...","[ACAO_A_EXECUTAR, EQUIPAMENTO, ACAO_A_EXECUTAR]","[0.9999902248382568, 1.0, 0.9997515678405762]"
1,ProcedimentosObjetivo / Item de Controle1---CO...,"[Verificar, indisponibilidade, LT 230 kV Tucur...","[ACAO_A_EXECUTAR, ESTADO_OPERATIVO, EQUIPAMENTO]","[0.9999995231628418, 0.9999918937683105, 0.999..."
2,2COSR-NCOCOSR-NCOEquatorialDesligar o religame...,"[religamento automático, LT 230 kV Altamira / ...","[ACAO_A_EXECUTAR, EQUIPAMENTO]","[0.9972758293151855, 0.9999115467071533]"
3,3COSR-NCOCOSR-NCOEletronorteAbrir o terminal d...,"[o terminal, LT 230 kV Tucuruí / Altamira,, SE...","[ACAO_A_EXECUTAR, EQUIPAMENTO, SUBESTACAO]","[0.9999996423721313, 1.0, 0.9858415722846985]"
4,- Reator da LT disponível:SE Tucuruí ≤ 232 kV-...,"[Reator da LT, SE Tucuruí, 232 kV-, Reator da ...","[EQUIPAMENTO, SUBESTACAO, VALOR_COM_UNID.MEDID...","[0.9989086389541626, 0.9999986290931702, 0.780..."
...,...,...,...,...
380,V ≤ 46 kV  ≤ 2003COSR-NCOCOSR-NCOCOS-Am GT (A...,"[46 kV, Amazonas, GT Normalizar o atendimento ...","[VALOR_COM_UNID.MEDIDA, SUBESTACAO, ACAO_A_EXE...","[0.9999377131462097, 0.7745931148529053, 0.999..."
381,4COSR-NCOCOSR-NCOCOS-Am GT (Amazonas GT Ligar ...,"[Ligar o religamento automático, LT 230 kV Bal...","[ACAO_A_EXECUTAR, EQUIPAMENTO]","[0.8638736009597778, 0.9999288320541382]"
382,Desenergização da LT 230 kV Cristiano Rocha / ...,"[Desenergização, LT 230 kV Cristiano Rocha / L...","[ACAO_A_EXECUTAR, EQUIPAMENTO, ACAO_A_EXECUTAR...","[0.9993795156478882, 0.9999955296516418, 0.816..."
383,- Atendida as limitações impostas pela indispo...,"[Atendida, indisponibilidade, LT 230 kV Cristi...","[ACAO_A_EXECUTAR, ESTADO_OPERATIVO, EQUIPAMENT...","[0.9999533891677856, 0.9999958276748657, 0.999..."


In [8]:
entity_sequence = df['type'].to_list()
# entity_sequence[0:2]

## Frequent Pattern

In [9]:
# #Finding the frequent patterns with min support threshold=0.5
# FrequentPatterns=pyfpgrowth.find_frequent_patterns(entity_sequence,support_threshold=0.5)
# print(FrequentPatterns)
 
# # Generating rules with min confidence threshold=0.5
# Rules=pyfpgrowth.generate_association_rules(patterns=FrequentPatterns,confidence_threshold=0.5)
# Rules

## RHS and LHS

In [10]:
rules_dict = {
    'rhs':[],
    'lhs':[],
    'support':[],
    'confidence':[],
    'lift':[]
}

rules = apriori(entity_sequence, min_support = 0.05, min_confidence = 0.58, min_lift=1.15, min_length = 2, max_length=3)
rules = list(rules)
rules

# Extracting rules from the object
for i in range(len(rules)):
    rules_dict['rhs'].append(list(rules[i][2][0][0]))
    rules_dict['lhs'].append(list(rules[i][2][0][1]))
    rules_dict['support'].append(rules[i][1])
    rules_dict['confidence'].append(rules[i][2][0][2])
    rules_dict['lift'].append(rules[i][2][0][3])

rules_df = pd.DataFrame(rules_dict)
rules_df.sort_values(by=['lift','confidence'], ascending=False, inplace=True)
rules_df.reset_index(drop=True, inplace=True)
rules_df.head(60)

,rhs,lhs,support,confidence,lift
0,[OPERADOR_MATEMATICO],"[SUBESTACAO, VALOR_COM_UNID.MEDIDA]",0.057143,0.846154,4.286437
1,[OPERADOR_MATEMATICO],[VALOR_COM_UNID.MEDIDA],0.064935,0.961538,2.914900
2,[OPERADOR_MATEMATICO],[SUBESTACAO],0.057143,0.846154,1.850962
3,[VALOR_COM_UNID.MEDIDA],[SUBESTACAO],0.197403,0.598425,1.309055
4,"[ACAO_A_EXECUTAR, SUBESTACAO]",[EQUIPAMENTO],0.283117,0.947826,1.232814
5,[ACAO_A_EXECUTAR],[EQUIPAMENTO],0.719481,0.917219,1.193004
6,"[SUBESTACAO, ESTADO_OPERATIVO]",[EQUIPAMENTO],0.051948,0.909091,1.182432


In [11]:
len(rules_df)

7

### Pattern example

In [12]:
df['entity'].iloc[0]

array(['Desenergização', 'LT 230 kV Tucuruí / Altamira', 'sentido único'],
      dtype=object)

In [13]:
df['type'].iloc[0]

array(['ACAO_A_EXECUTAR', 'EQUIPAMENTO', 'ACAO_A_EXECUTAR'], dtype=object)

## Raw sequence

In [29]:
rules_dict = {
    'sequence':[],
    'support':[],
    'confidence':[],
    'lift':[]
}

association_rules = apriori(entity_sequence, min_support = 0.05, min_confidence = 0.55, min_lift=1.10, min_length = 2, max_length=8)
association_rules = list(association_rules)

for item in association_rules:
    rules_dict['sequence'].append(list(item[0]))
    rules_dict['support'].append(item[1])
    rules_dict['confidence'].append(item[2][0][2])
    rules_dict['lift'].append(item[2][0][3])

sequence_df = pd.DataFrame(rules_dict)
sequence_df.sort_values(by=['lift','support','confidence'], inplace=True, ascending=False)
sequence_df.reset_index(drop=True, inplace=True)
sequence_df.head(20)

,sequence,support,confidence,lift
0,"[SUBESTACAO, VALOR_COM_UNID.MEDIDA, OPERADOR_M...",0.057143,0.846154,4.286437
1,"[VALOR_COM_UNID.MEDIDA, OPERADOR_MATEMATICO]",0.064935,0.961538,2.914900
2,"[SUBESTACAO, OPERADOR_MATEMATICO]",0.057143,0.846154,1.850962
3,"[SUBESTACAO, VALOR_COM_UNID.MEDIDA]",0.197403,0.598425,1.309055
4,"[ACAO_A_EXECUTAR, SUBESTACAO, EQUIPAMENTO]",0.283117,0.947826,1.232814
5,"[ACAO_A_EXECUTAR, EQUIPAMENTO]",0.719481,0.917219,1.193004
6,"[SUBESTACAO, EQUIPAMENTO, ESTADO_OPERATIVO]",0.051948,0.909091,1.182432
7,"[ACAO_A_EXECUTAR, EQUIPAMENTO, VALOR_COM_UNID....",0.132468,0.879310,1.143698
8,"[ACAO_A_EXECUTAR, EQUIPAMENTO, ESTADO_OPERATIVO]",0.158442,0.871429,1.133446


In [30]:
sequence_df['sequence'].iloc[4]

['ACAO_A_EXECUTAR', 'SUBESTACAO', 'EQUIPAMENTO']

## Examples

In [28]:
ex_seq = sequence_df['sequence'].iloc[4]
for index, row in df.iterrows():
    if all(item in row['type'] for item in ex_seq):
        print(' text: \n',row['text'])
        print(' entities: \n',row['entity'])
        print(' types: \n',row['type'])
        print('-'*10)

 text: 
 3COSR-NCOCOSR-NCOEletronorteAbrir o terminal da LT 230 kV Tucuruí / Altamira, na SE Altamira.
 entities: 
 ['o terminal' 'LT 230 kV Tucuruí / Altamira,' 'SE Altamira.']
 types: 
 ['ACAO_A_EXECUTAR' 'EQUIPAMENTO' 'SUBESTACAO']
----------
 text: 
 - Reator da LT disponível:SE Tucuruí ≤ 232 kV- Reator da LT indisponível:SE Tucuruí ≤ 220 kVDesenergizar a LT 230 kV Tucuruí / Altamira, pelo terminal da SE Tucuruí.
 entities: 
 ['Reator da LT' 'SE Tucuruí' '232 kV-' 'Reator da LT indisponível:'
 'SE Tucuruí' '220' 'LT 230 kV Tucuruí / Altamira,' 'pelo terminal da'
 'SE Tucuruí.']
 types: 
 ['EQUIPAMENTO' 'SUBESTACAO' 'VALOR_COM_UNID.MEDIDA' 'ESTADO_OPERATIVO'
 'SUBESTACAO' 'VALOR_COM_UNID.MEDIDA' 'EQUIPAMENTO' 'ACAO_A_EXECUTAR'
 'SUBESTACAO']
----------
 text: 
 Energização da LT 230 kV Tucuruí / Altamira (sentido único PassoCoordenaçãoControleComandoProcedimentosLimite / Item de Controle1COSR-NCOCOSR-NCOEletronorteEnergizar, a partir da SE Tucuruí, a LT 230 kV Tucuruí / Altamira.
 e